# Laboratorio 7.2: Redes bayesianas

En el laboratorio aprenderemos a cargar una red bayesiana y a calcular probabilidades a partir de evidencia. Además, construiremos la estructura de la red a partir de los datos e implementaremos un clasificador.

## A. Inferencia

## 1. Cargamos las librerías para trabajar con R

In [ ]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## 2. Cargamos las librerías bnlearn y bnclassify

Las librerías `bnlearn` son necesarias para trabajar con redes bayesianas.

In [ ]:
%%R

# Cargamos libreria de redes bayesianas
install.packages("bnlearn")
library(bnlearn)

# Librerias para clasificacion con redes bayesianas
install.packages("bnclassify")
library(bnclassify) 

In [ ]:
%%R
library(foreign) # Lee archivos SPSS

### 3. Cargamos el archivo desde Google Drive

Cargamos la base de datos que se encuentra guardada en un archivo SPSS.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%R
# Borrar las variables del ambiente
rm(list=ls())

## Fijar mi path raiz
setwd('/content/drive/My Drive/D-UCN/Projects/2020/Propuestos/Psicologia-DeterioroCognitivo/Data/DatosOrientacionMemoria-20200629')
name.file <- 'Base FONIS orietacion memoria.sav'

dataset <- read.spss(name.file, to.data.frame=TRUE)

names(dataset)

### 4. Detectamos si existen archivos *missing*

Si existen campos que no tienen valores

In [ ]:
%%R
# Detectar filas missing y eliminar
df_clear_id <- na.omit(dataset)
print(sprintf("Número de filas sin missing: %d",nrow(df_clear_id)))
print(sprintf("Número de filas perdidas: %d",nrow(dataset) - nrow(df_clear_id)))
anyNA(df_clear_id)

### 5. Elimino el identificador ID

El identificador ID es solo un número correlativo y no  es necesario para la clasificación



In [ ]:
%%R
# Elimino el identificador 
df.clear <- df_clear_id[, !(names(df_clear_id) %in% c('ID'))]

print(names(df.clear))

In [ ]:
%%R
!(names(df_clear_id) %in% c('ID'))

### 6. Transformo las columnas numéricas a factores

Para que la red bayesiana sea reconocida como discreta debo dejar sus valores como factores.

`lapply()` intentará coercionar a una lista el objeto que demos como argumento y después aplicará una función a todos sus elementos.

In [ ]:
%%R
# Transformaciones de las columnas a factores
col.names <- names(df.clear)
df.clear[, col.names] <- lapply(df.clear[, col.names], factor)

In [ ]:
%%R
str(df.clear)

### 7. Elimino aquellas columnas que no son necesarias para el experimento



In [ ]:
%%R
drops <- c('GDS', 'GDS_R2', 'GDS_R2')
df.clear <- df.clear[, !(names(df.clear) %in% drops)]
df.clear

In [ ]:
%%R
!(names(df.clear) %in% drops)

### 8. Construir estructura de la red

Tree Augmented Naive (Tree Augmented Network), es un grafo que permite crear un clasificador Naïve.

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("graph")
BiocManager::install("Rgraphviz")

In [ ]:
%%R
library("Rgraphviz")
# Clasificacion 
# Construir la estructura de la red
tan.detcog <- tan_cl(class='GDS_R1', df.clear)
plot(tan.detcog)

Proceso de aprendizaje de la red.

In [ ]:
%%R
tan.detcog <- lp(tan.decog, df.clear, smooth= 0.01)

### 9. Predicción

Probabilidades

In [ ]:
%%R
pred.prob.tn <- predict(tan.detcot, df_clear, prob=TRUE)

Clases

In [ ]:
%%R
pred.class.tn <- predict(tan.detcot, df_clear, prob=FALSE)

### 10. Guardo los resultados en formato tabla

In [ ]:
%%R
db.tan <- cbind(df.clear, pred.prob.tn, pred.class.tn)

### 11. Calculo la matriz de confusión


In [ ]:
%%R
cm.tan <- table(predicted=pred.class.tn, true=df.clear$GDS_R1)
n = sum(cm.tan) # Número de instacias
nc = nrow(cm.tan) # Numero de clases
diag = diag(cm.tan) # Número de clases correctamente clasificadas
rowsum = apply(cm.tan, 1, sum) # Número de instancias por clase
colsum = apply(cm.tan, 2, sum) # Número de predicciones por clase

#### 12. Calculamos algunas métricas


In [ ]:
%%R 
accuracy = sum(diag) / n
precision = diag / colsum
recall = dia / rowsums
f1 = 2 * precision * recall / (precision + recall)

print(data.frame(precision, recall, f1))

In [ ]:
%%R
# Otra forma de calcular las métricas
print(bnclassify::accuracy(pred.class.tn, df.clear$GDS_R1))

### 13. Guardar los datos

In [ ]:
%%R

## Guardar los resultados en un csv
db.tan.id <- cbind(df_clear_id$ID, db.tan)
write.csv2(db.tan.id, "archivo.csv", row.names=TRUE)